#### 使用fasttext工具实现word2vec的训练和使用

- 第一步:获取训练数据
- 第二步:训练词向量
- 第三步:模型超参数设定
- 第四步:模型效果检验
- 第五步:模型的保存与重加载

#### 第一步:获取训练数据

-  wget -c http://mmattmahoney.net/dc/enwik9.zip -P data
- unzip data/enwik9.zip -d data
- Head -10 data/enwik9
- Perl wikifil.pl data/enwik9 > data/fil9
- Head -c 80 data/fil9

In [ ]:
import fileinput

import fasttext
# 使用fasttext的train_unsupervised(无监督训练方法)进行词向量的训练
model = fasttext.train_unsupervised('data/fil9')

# 有效训练词汇量为124M,共218316个单词

# 通过get_word_vector方法来获得指定词汇的词向量
model.get_word_vector("the")

#### 第三步：模型超参数的设定

- 在训练词向量过程中，我们可以设定很多常用超参数来调节我们模型效果如:
- 无监督训练模式：skipgram或者cbow，默认是skipgram,在实践中,skipgram模式在利用子词方面比cbow更好
- 词嵌入维度dim:默认为100，但随着语料库的增大，词嵌入的维度往往也要更大
- 学习率lr:默认是0.05,根据经验，建议选择[0.01, 1]范围内
- 使用线程数thread：默认是12个线程，一般建议和CPU核心数相同

In [ ]:
model = fasttext.train_unsupervised('data/fil9', "cbow", dim=300, epoch=1, lr=0.1, thread=8)

#### 第四步：模型效果检验
- 检查单词向量质量的一种简单的方法就是查看其邻近单词，通过我们主观判断这些单词是否与目标单词相关程度来粗略评定模型效果好坏

In [ ]:
#查找”运动“的邻近单词，我们可以发现”体育网“."运动汽车"，”运动服“等
model.get_nearest_neighbors("sports")
# 查找"音乐"的邻近单词，我们可以发现音乐有关的词汇
model.get_nearest_neighbors("music")

# 查找”小狗“的邻近单词，我们可以发现小狗有关的词汇
model.get_nearest_neighbors('dog')

#### 第五步:模型的保存与重加载

In [ ]:
# 使用save_model保存模型
model.save_model('fil9.bin')

# 使用fasttext.laod_model加载模型
model = fasttext.load_model('fil9.bin')
model.get_word_vector("the")

#### 什么是word embedding(词嵌入)
- 通过一定的方式将词汇映射到指定维度(一般是更高维度)的空间
- 广义的word embedding包括所有密集词汇向量的表示方法，如之前学习的word2vec,即可认为是word embedding的一种
- 狭义的word embedding是指神经网络中加入的embedding层，对整个网络进行训练的同时产生embedding矩阵(embedding层的参数)，这个embedding矩阵就是训练过程中所有输入词汇的向量表示组成的矩阵

In [ ]:
# 导入torch和tensorboard导入进来
import torch
import fileinput
from torch.utils.tensorboard import SummaryWriter

# 实例化一个写入对象
writer = SummaryWriter()

# 随机初始化一个100*5的矩阵, 将其视作已经得到的词嵌入矩阵
embedded = torch.randn(100, 50)

# 导入事先准备好的100个中文词汇文件, 形成meta列表原始词汇
meta = list(map(lambda x: x.strip(), fileinput.FileInput("./vocab100.csv")))
writer.add_embedding(embedded, metadata=meta)
writer.close()


#### 在终端启动tensorboard服务
- tensorboard --logdir runs --host 0.0.0.0
- 通过http://0.0.0.0:6006访问浏览器可视化页面

#### 小结

##### 学习什么是文本张量表示

- 将一段文本使用张量进行表示，其中一般将词汇表示成向量:称作词向量，再由各个词向量按顺序组成矩阵形成文本表示

##### 学习文本张量表示的作用

-  将文本表示成张量(矩阵)形式，能够使语言文本可以作为计算机处理程序的输入，进行接下来的一系列的解析工作

##### 学习文本张量表示的方法

- one-hot编码
- Word2vec
- Word Embedding

##### 学习了什么是one-hot词向量表示

-  又称热独编码，将每个词表示成具有n个元素的向量，这个词向量中只有一个元素是1，其他元素都是0，不同词汇元素为0的位置不同，其中n的大小是整个语料中不同词汇的总数
- one-hot编码的优劣势
	- 优势：操作简单，容易理解
	- 劣势：完全割裂了词与词之间的联系，而且在语料集下，每个向量和长度过长，占据大量内存

#### 学习了什么是word2vec

- 是一种流行的将词汇表示成向量的无监督训练方法，该过程将构建神经网络模型，将网络参数作为词汇的向量表示，它包含CBOW和skipgram两种训练模式
- CBOW(Continuous bag of words)模式
	- 给定一段用于训练的文本语料，再选定某段长度(窗口)作为研究对象，使用上下文词汇预测目标词汇
	- 假设我们给定的训练语料只有一句话:Hope can set you free(愿你自由成长)，窗口大小为3，因此模型的第一个训练样本来自Hope you set,因为是CBOW模式，所以将使用Hope和set作为输入，you作为输出，在模型训练时，Hope,set,you等词汇都使用它们的one-hot编码，如图所示:每个one-hot编码的单词与各自的变换矩阵(即参数矩阵3x5,这里的3是指最后得到的词向量维度)相乘之后再相加，得到上下文表示矩阵(3x1)
	- 接着，将上下文表示矩阵与变换矩阵(参数矩阵5x3，所的有变换矩阵共享参数)相乘，得到5x1的结果矩阵，它将与我们真正的目标矩阵即you的one-hot编码矩阵(5x1)进行损失的计算，然后更新网络参数完成一次模型迭代
	- 最后窗口按序向后移动，重新更新参数，直到所有的语料被遍历完成 ，得到最终的变换矩阵(3x5),这个变换矩阵与每个词汇的one-hot编码(5x1)相乘，得到3x1的矩阵就是该词汇的word2vec张量表示
- skipgram模式
	- 给定一段用于训练的文本语料，再选定某段长度(窗口)作为研究对象，使用目标词汇预测上下文词汇
	-  假设我们给定的训练语料只有一句话:Hope can set you free(愿你自由成长)，窗口大小为3，因此模型的第一个训练样本来自Hope you set，因为是skipgram模式，所以将使用you作为输入，hope和set作为输出,在模型训练时，Hope，set，you等词汇都使用它们的one-hot编码，将you的one-hot编码与变换矩阵(即参数矩阵3x5,这里的3是指最后得到的词向量维度)相乘，得到目标词汇表示矩阵(3x1)
	- 接着，将目标词汇表示矩阵与多个变换矩阵(参数矩阵5x3)相乘，得到5x1的结果矩阵，它将与我们hope和set对应的one-hot编码矩阵(5x1)进行损失的计算，然后更新网络参数完成一次模型迭代
	- 最后窗口按序向后移动，重新更新参数，直到所有语料被遍历完成，得到最终的变换矩阵即参数矩阵(3x5).这个变换矩阵与每个词汇的one-hot编码(5x1)相乘，得到的3x1的矩阵就是该词汇的word2vec张量表示
